In [1]:
import os,sys

In [2]:
os.environ["SPARK_HOME"] = "C:/spark"
os.environ["PYLIB"] ="C:/spark/python/lib"
sys.path.insert(0,os.environ["PYLIB"] + "/py4j-0.10.4-src.zip" )
sys.path.insert(0,os.environ["PYLIB"] + "/pyspark.zip" )

In [3]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
sc = SparkContext()

In [4]:
spark_session = SparkSession(sc).builder.appName("Machine Learning Demo").master('local').getOrCreate()
spark_session

In [14]:
# Read and split data in 80:20 ratio's
data = spark_session.read.format("libsvm").load("visitors_demo.txt")
train, test = data.randomSplit([0.8, 0.2], seed=12345)

In [15]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression()

In [16]:
# Fit the model
lrModel = lr.fit(train)
trainingSummary = lrModel.summary

In [17]:
trainingSummary

In [18]:
trainingSummary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[21.6317654290919...|[0.99999999959687...|       0.0|
|  0.0|(692,[98,99,100,1...|[29.1669451144556...|[0.99999999999978...|       0.0|
|  0.0|(692,[100,101,102...|[19.5384623182738...|[0.99999999672995...|       0.0|
|  0.0|(692,[121,122,123...|[20.4308944645952...|[0.99999999866039...|       0.0|
|  0.0|(692,[122,123,148...|[20.8725477109517...|[0.99999999913867...|       0.0|
|  0.0|(692,[123,124,125...|[31.8576743192384...|[0.99999999999998...|       0.0|
|  0.0|(692,[124,125,126...|[21.6271328593757...|[0.99999999959499...|       0.0|
|  0.0|(692,[124,125,126...|[31.1364217577289...|[0.99999999999997...|       0.0|
|  0.0|(692,[124,125,126...|[28.2664031423809...|[0.99999999999947...|       0.0|
|  0.0|(692,[124

In [19]:
from pyspark.mllib.evaluation import MulticlassMetrics
lrModel.evaluate(test)
predictionAndLabels = lrModel.evaluate(test)

In [20]:
predictionAndLabels.predictions.show()

predictionAndLabels = predictionAndLabels.predictions.select('label','prediction')


predictionAndLabels.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[122,123,124...|[16.5552884752723...|[0.99999993541531...|       0.0|
|  0.0|(692,[123,124,125...|[29.4623128061463...|[0.99999999999983...|       0.0|
|  0.0|(692,[123,124,125...|[18.7544818647511...|[0.99999999283803...|       0.0|
|  0.0|(692,[125,126,127...|[23.6186440119655...|[0.99999999994472...|       0.0|
|  0.0|(692,[126,127,128...|[27.7382587359914...|[0.99999999999910...|       0.0|
|  0.0|(692,[126,127,128...|[21.2554598059638...|[0.99999999941268...|       0.0|
|  0.0|(692,[153,154,155...|[29.6607503169545...|[0.99999999999986...|       0.0|
|  0.0|(692,[155,156,180...|[26.0487784075924...|[0.99999999999513...|       0.0|
|  0.0|(692,[234,235,237...|[5.32088955575381...|[0.99513537695598...|       0.0|
|  1.0|(692,[123

In [21]:
# Evaluate Model using Binary Classificationn 

In [22]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator

evaluator = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')

# For multiclass
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label',
                                             metricName='accuracy')

# Evaluate Accuracy of the model 
acc = evaluator.evaluate(predictionAndLabels)



1.0

In [25]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
# Evaluate F1 score for the model
f1 = evaluator.evaluate(predictionAndLabels)
f1

1.0